In [43]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
import torch.optim as optim

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
def load_graph(file_path):
    edgelist = []
    set_a = []
    set_b = []

    with open(file_path) as f:
        for ind, line in enumerate(f):
            if ind == 0: 
                na, nb, ns = map(int, line.split('\t'))
                set_b = [f"set_b_{i}" for i in range(nb)]
                set_a = [f"set_a_{i}" for i in range(na)]
                continue
            a, b, s = map(int, line.split('\t'))
            if s == 1:
                edgelist.append((f"set_a_{a}", f"set_b_{b}", {'sign': '+1'}))
            else:
                edgelist.append((f"set_a_{a}", f"set_b_{b}", {'sign': '-1'}))

    B = nx.Graph()
    B.add_nodes_from(set_a, bipartite=0)
    B.add_nodes_from(set_b, bipartite=1)
    B.add_edges_from(edgelist)
    
    return B

In [46]:
def build_drawing(B, set_a, set_b):

    colors = ['yellow' for _ in set_a] + ['green' for _ in set_b]
    
 
    num_nodes = B.number_of_nodes()
    print("Number of nodes in the graph:", num_nodes)
    

    edge_colors = []
    for u, v, attrs in B.edges(data=True):
        if attrs['sign'] == '+1':
            edge_colors.append('blue')   
        else:
            edge_colors.append('red')   
    

    nx.draw(B, with_labels=False, node_color=colors, edge_color=edge_colors)
    plt.show()


In [88]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_prob=0.4):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)   
        self.dropout = nn.Dropout(p=dropout_prob)   
        self.output_layer = nn.Linear(output_dim, output_dim)  

    def forward(self, x):
        x = F.relu(self.fc1(x))    
        x = self.dropout(x)       
        x = self.output_layer(x)   
        return x

In [69]:
class SBGNN(nn.Module):
    def __init__(self, graph, emb_dim_a=32, emb_dim_b=32):
        super(SBGNN, self).__init__()

        self.graph = graph
        self.set_a = [n for n, d in self.graph.nodes(data=True) if d['bipartite'] == 0]
        self.set_b = [n for n, d in self.graph.nodes(data=True) if d['bipartite'] == 1]
        self.emb_dim_a = emb_dim_a
        self.emb_dim_b = emb_dim_b

        # Initialize ModuleDict for features_a and features_b
        self.features_a = nn.ModuleDict({str(node): nn.Embedding(1, self.emb_dim_a) for node in self.set_a})
        self.features_b = nn.ModuleDict({str(node): nn.Embedding(1, self.emb_dim_b) for node in self.set_b})

        self.weight_a_b = nn.Parameter(torch.randn(self.emb_dim_a, self.emb_dim_a))
        self.weight_a_u = nn.Parameter(torch.randn(self.emb_dim_a, self.emb_dim_a))
        self.weight_b_b = nn.Parameter(torch.randn(self.emb_dim_b, self.emb_dim_b))
        self.weight_b_u = nn.Parameter(torch.randn(self.emb_dim_b, self.emb_dim_b))

        self.mlp_a = MLP(3 * self.emb_dim_a, self.emb_dim_a)
        self.mlp_b = MLP(3 * self.emb_dim_b, self.emb_dim_b)

    def message_passing(self, iterations=1):
        for i in range(iterations):
            new_features_a = {}
            new_features_b = {}

            for node in self.set_a:
                new_features_a_b = torch.zeros(self.emb_dim_a,device=device)
                new_features_a_u = torch.zeros(self.emb_dim_a,device=device)
                count_b = 0
                count_u = 0

                for neighbor in self.graph.neighbors(node):
                        sign = self.graph.edges[node, neighbor]['sign']
                        if sign == '+1':
                            new_features_a_b += self.features_b[str(neighbor)].weight.squeeze(0)
                            count_b += 1
                        else:
                            new_features_a_u += self.features_b[str(neighbor)].weight.squeeze(0)
                            count_u += 1

                if count_b > 0:
                    new_features_a_b /= count_b
                    new_features_a_b = torch.matmul(new_features_a_b, self.weight_a_b)

                if count_u > 0:
                    new_features_a_u /= count_u
                    new_features_a_u = torch.matmul(new_features_a_u, self.weight_a_u)

                new_features_a[str(node)] = torch.cat((new_features_a_b, new_features_a_u, self.features_a[str(node)].weight.squeeze(0)), dim=0)
                new_features_a[str(node)] = self.mlp_a(new_features_a[str(node)])

            for node in self.set_b:
                new_features_b_b = torch.zeros(self.emb_dim_b,device=device)
                new_features_b_u = torch.zeros(self.emb_dim_b,device=device)
                count_b = 0
                count_u = 0

                for neighbor in self.graph.neighbors(node):
                        sign = self.graph.edges[node, neighbor]['sign']
                        if sign == '+1':
                            new_features_b_b += self.features_a[str(neighbor)].weight.squeeze(0)
                            count_b += 1
                        else:
                            new_features_b_u += self.features_a[str(neighbor)].weight.squeeze(0)
                            count_u += 1

                if count_b > 0:
                    new_features_b_b /= count_b
                    new_features_b_b = torch.matmul(new_features_b_b, self.weight_b_b)

                if count_u > 0:
                    new_features_b_u /= count_u
                    new_features_b_u = torch.matmul(new_features_b_u, self.weight_b_u)

                new_features_b[str(node)] = torch.cat((new_features_b_b, new_features_b_u, self.features_b[str(node)].weight.squeeze(0)), dim=0)
                new_features_b[str(node)] = self.mlp_b(new_features_b[str(node)])

            # Update ModuleDicts
            for node in self.set_a:
                self.features_a[str(node)].weight = nn.Parameter(new_features_a[str(node)].unsqueeze(0))

            for node in self.set_b:
                self.features_b[str(node)].weight = nn.Parameter(new_features_b[str(node)].unsqueeze(0))

    


In [70]:
class Softmax(nn.Module):
    def __init__(self, input_dim, cluster_number):
        super(Softmax, self).__init__()
        self.fc = nn.Linear(input_dim, cluster_number)  # Fully connected layer

    def forward(self, x):
        x = self.fc(x)         # Apply the fully connected layer
        x = F.softmax(x, dim=0)  # Apply softmax to the output
        return x


In [71]:
class Community(nn.Module):
    def __init__(self, sbgnn, cluster_number):
        super(Community, self).__init__()
        
        self.sbgnn=sbgnn
        self.cluster_number=cluster_number
        self.emb_dim_a=self.sbgnn.emb_dim_a
        self.emb_dim_b=self.sbgnn.emb_dim_b

        self.Softmax_a=Softmax(self.emb_dim_a,self.cluster_number)
        self.Softmax_b=Softmax(self.emb_dim_b,self.cluster_number)


    def build_communities(self):
        self.list_a=[]
        self.list_b=[]
        
        for node in self.sbgnn.set_a:
            new_tensor=self.sbgnn.features_a[str(node)].weight.squeeze(0)
            new_tensor=self.Softmax_a(new_tensor)
            self.list_a.append(new_tensor)
        
        self.C_a=torch.stack(self.list_a,dim=0).squeeze(0)
        self.C_a=self.C_a.to(dtype=torch.float64)


        for node in self.sbgnn.set_b:
            new_tensor=self.sbgnn.features_b[str(node)].weight.squeeze(0)
            new_tensor=self.Softmax_b(new_tensor)
            self.list_b.append(new_tensor)
        
        self.C_b=torch.stack(self.list_b,dim=0).squeeze(0)
        self.C_b=self.C_b.to(dtype=torch.float64)

In [118]:
class Modularity(nn.Module):
    def __init__(self,sbgnn,community):
        super(Modularity, self).__init__()

        self.graph=sbgnn.graph
        self.set_a = sbgnn.set_a
        self.set_b = sbgnn.set_b
        self.n=len(self.set_a)
        self.m=len(self.set_b)

        self.community=community
        self.C_a=self.community.C_a
        self.C_b=self.community.C_b
        self.k=self.community.cluster_number

        self.positive_counts={node:0 for node in self.graph.nodes()}
        self.negative_counts={node:0 for node in self.graph.nodes()}
        self.total_positive=0
        self.total_negative=0


    def build_adjacency_matrix(self):
        matrix = np.zeros((self.n, self.m))
        for i, a_node in enumerate(self.set_a):
            for j, b_node in enumerate(self.set_b):
                if self.graph.has_edge(a_node, b_node) and self.graph[a_node][b_node]['sign']=='+1':
                    matrix[i, j] = 1
                elif self.graph.has_edge(a_node, b_node) and self.graph[a_node][b_node]['sign']=='-1':
                    matrix[i,j] = -1
        self.adj_matrix=torch.from_numpy(matrix).to(device)


    def count_edges(self):
        for u, v, data in self.graph.edges(data=True):
            if data['sign'] =='+1':
                self.positive_counts[u] += 1
                self.positive_counts[v] += 1
                self.total_positive+=1
            elif data['sign'] =='-1':
                self.negative_counts[u] += 1
                self.negative_counts[v] += 1
                self.total_negative+=1
        
        self.positive_plus_negative=self.total_positive+self.total_negative
    
    def build_W_matrix(self):
        matrix=[]
        for node_i in self.set_a:
            temp_list=[]
            for node_j in self.set_b:
                pos=(self.positive_counts[node_i]*self.positive_counts[node_j])/self.total_positive
                neg=(self.negative_counts[node_i]*self.negative_counts[node_j])/self.total_negative
                temp_list.append(pos-neg)
            matrix.append(temp_list)
        matrix=np.matrix(matrix)
        matrix = self.adj_matrix.cpu().numpy() - matrix
        self.W_matrix=torch.from_numpy(matrix).to(device)

    def find_modularity(self):
        denominator=1/self.positive_plus_negative
        final_matrix= torch.matmul(self.C_a.t(),self.W_matrix)
        self.final_matrix=torch.matmul(final_matrix,self.C_b)
        self.trace=torch.trace(self.final_matrix)
        self.modularity=denominator*self.trace


    def loss_function(self):

        self.soft_count_a=torch.sum(self.C_a,dim=0)
        self.soft_count_b=torch.sum(self.C_b,dim=0)

        self.frobenius_norm_a=torch.norm(self.soft_count_a,p='fro')
        self.frobenius_norm_b=torch.norm(self.soft_count_b,p='fro')

        self.reg_a=(np.sqrt(self.k)/self.n)*self.frobenius_norm_a
        self.reg_b=(np.sqrt(self.k)/self.m)*self.frobenius_norm_b

        self.regularization=self.reg_a+self.reg_b -2

        self.loss= self.regularization-self.modularity 

In [122]:
def train_sbgnn(sbgnn, community, lr=0.01, epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    sbgnn.to(device)
    community.to(device)
    optimizer = optim.Adam(list(sbgnn.parameters()) + list(community.parameters()), lr=lr)
    for epoch in range(epochs):
        optimizer.zero_grad()
        sbgnn.message_passing(iterations=2)
        community.build_communities()
        modularity=Modularity(sbgnn,community)
        modularity.to(device)
        modularity.build_adjacency_matrix()
        modularity.count_edges()
        modularity.build_W_matrix()
        modularity.find_modularity()
        modularity.loss_function()

        loss = modularity.loss
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            print(f"Modularity : {modularity.modularity}")


In [123]:
file_path = r"C:\Users\rishi\ml_projects\SBGNN\senate1to10.txt"
B = load_graph(file_path)
# set_a = [n for n, d in B.nodes(data=True) if d['bipartite'] == 0]
# set_b = [n for n, d in B.nodes(data=True) if d['bipartite'] == 1]
#build_drawing(B,set_a,set_b)

In [124]:
sbgnn=SBGNN(B,32,32)
community=Community(sbgnn,20)

train_sbgnn(sbgnn,community)

Epoch 0, Loss: 0.03427046178521733
Modularity : 9.130466987828017e-07
Epoch 1, Loss: 0.02431806987839236
Modularity : 5.683600129394359e-07
Epoch 2, Loss: 0.026534582011598366
Modularity : 2.2398995617692435e-06
Epoch 3, Loss: 0.026242184289507683
Modularity : -5.473046525915146e-06
Epoch 4, Loss: 0.04293806321631391
Modularity : -2.3369250226082926e-05
Epoch 5, Loss: 0.052336137959806704
Modularity : -1.0075599805668753e-05
Epoch 6, Loss: 0.045062276171896336
Modularity : -8.197687740787586e-06


KeyboardInterrupt: 